## Convert DES Data to SACC

In this example we will load a DES-format 2pt FITS file and convert it into a Sacc file.

This requires the DES twopoint library, which is available using `pip install twopoint`.

In [1]:
pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import twopoint
import sacc
from twopoint import Types
from sacc import Sacc, standard_types
import os

Download the DES data file.

In [3]:
import urllib.request
filename = '2pt_NG_mcal_1110.fits'
url = 'http://desdr-server.ncsa.illinois.edu/despublic/y1a1_files/chains/2pt_NG_mcal_1110.fits'
if os.path.exists(filename):
    print("File already downloaded")
else:
    print("Downloading file")
    urllib.request.urlretrieve(url, filename)
    print("Done")


File already downloaded


Load in the DES data file, and make a new empty sacc file


In [4]:
T = twopoint.TwoPointFile.from_fits(filename)
S = Sacc()

This table converts the type codes used in twopoint to sacc names


In [5]:
types = {
    (Types.galaxy_shear_plus_real, Types.galaxy_shear_plus_real): standard_types.galaxy_shear_xi_plus,
    (Types.galaxy_shear_minus_real, Types.galaxy_shear_minus_real): standard_types.galaxy_shear_xi_minus,
    (Types.galaxy_position_real, Types.galaxy_shear_plus_real): standard_types.galaxy_shearDensity_xi_t,
    (Types.galaxy_position_real, Types.galaxy_position_real): standard_types.galaxy_density_xi,
}

In TwoPoint n(z) kernels are grouped into general types,
whereas in sacc we make a new tracer for each bin.
This dict maps this name change.


In [6]:
tracer_names = {
    'nz_lens': 'lens_{}',
    'nz_source': 'source_{}',
}

First convert the DES kernel objecs -> Sacc tracers


In [7]:
for kernel in T.kernels:
    fmt = tracer_names[kernel.name]
    z = kernel.z
    for i in range(kernel.nbin):
        nz = kernel.nzs[i]
        # In DES we used 1-based bin indices.
        # Because I was young and reckless then.
        name = fmt.format(i)
        S.add_tracer('NZ', name, z, nz)

Now extract points from the spectrum objects and add them to the sacc data set


In [8]:
for s in T.spectra:
    data_type = types[(s.type1,s.type2)]
    fmt1 = tracer_names[s.kernel1]
    fmt2 = tracer_names[s.kernel2]
    n = len(s)
    for k in range(n):
        i = s.bin1[k] - 1
        j = s.bin2[k] - 1
        tracers = (fmt1.format(i), fmt2.format(j))
        S.add_data_point(data_type, tracers, s.value[k], theta=s.angle[k], theta_bin=s.angular_bin[k])

And finally add the covariance

In [9]:
S.add_covariance(T.covmat)

Now we have fully assembled the Sacc file we can save it to disc.

In [10]:
S.save_fits('des-converted.sacc', overwrite=True)